# Effective Tax Rate

# Setup

Run this cell **once** if you are in Google Colab:

In [ ]:
import sys

if "google.colab" in sys.modules:
    !rm -rf sample_data
    !git clone https://github.com/HMellor/personal-finance-uk.git
    !mv personal-finance-uk/* .
    !rm -r personal-finance-uk

    from google.colab import output

    output.enable_custom_widget_manager()

Then you can install the Python requirements:

In [ ]:
%pip install -q -r requirements.txt

## Interactive Figure

In [ ]:
from utils.ui import InteractiveFigure
import numpy as np
import pandas as pd
from plotly.graph_objects import FigureWidget
import plotly.express as px
from utils.loan import student_loan_repayment
from utils.tax import income_tax, national_insurance


def update(plan: str):
    data = pd.DataFrame()
    gross_income = pd.Series(np.arange(start=0, stop=150000, step=100))
    _income_tax = gross_income.apply(income_tax)
    _national_insurance = gross_income.apply(national_insurance)
    _student_loan = gross_income.apply(student_loan_repayment, plan=plan)

    data["effective tax"] = _income_tax + _national_insurance
    data["effective tax"][1:] /= gross_income[1:]
    data["effective tax after loan"] = _income_tax + _national_insurance + _student_loan
    data["effective tax after loan"][1:] /= gross_income[1:]

    data["net income"] = gross_income - _income_tax - _national_insurance
    data["net income after loan"] = (
        gross_income - _income_tax - _national_insurance - _student_loan
    )
    data.index = gross_income
    return data


def plot(fig: FigureWidget, data: pd.DataFrame):
    # Plot lines
    lines = px.line(data, x=data.index, y=data.columns)
    with fig.batch_update():
        # Add/update data
        if not fig.data:
            fig.update_layout(
                title_text="Effective tax rate of all taxes combined", title_x=0.5
            )
            fig.update_xaxes(tickprefix="£", nticks=15)
            fig.update_yaxes(title="Effective tax rate", tickformat=".1%", col=1)
            fig.update_yaxes(
                title="Net income", tickprefix="£", tickformat=".3s", col=2
            )
            fig.add_traces(lines.data, rows=[1, 1, 1, 1], cols=[1, 1, 2, 2])
        else:
            for old_data, new_data in zip(fig.data, lines.data):
                old_data.x = new_data.x
                old_data.y = new_data.y


InteractiveFigure(plot=plot, update=update, x_title="Gross income").app